# Import potrebných knižníc

In [3]:
import os
from datetime import datetime
from PIL import Image, ImageDraw
import numpy as np

# Kombinovanie GOLD dát

V prípade že počas pol hodiny existuje viacero obrázkov GOLD dát

Táto sitácia nastáva hlavne pri stanici Arecibo no aplikujeme tento kód na všetky dáta

In [11]:
#určenie a vytvorenie priečinkov
input_folder = "GOLD_jicamarca_no_white"
output_folder = "GOLD_jicamarca_combined"

os.makedirs(output_folder, exist_ok=True)

In [12]:
#definovanie funkcie pre získanie timestampu 
def parse_timestamp(filename):
    parts = filename.split("_")
    timestamp = parts[:5]
    return datetime(*map(int, timestamp))

In [ ]:
image_groups = {}
#pre každý obrázok
for filename in os.listdir(input_folder):
    #kontrola či nie sú v priečinku aj iné súbory
    if filename.endswith(".png"):
        #získanie časových údajov
        timestamp = parse_timestamp(filename)
        minute = timestamp.minute
        #roztriedenie na pol hodinové intervalové skupiny
        half_hour_segment = '00' if minute < 30 else '30'
        hour_key = timestamp.strftime(f"%Y_%m_%d_%H_{half_hour_segment}")
        if hour_key not in image_groups:
            image_groups[hour_key] = []
        image_groups[hour_key].append(os.path.join(input_folder, filename))

#pre každú skpinu a všetky snímky v nej
for hour_key, filenames in image_groups.items():
    images = [Image.open(filename).convert("RGB") for filename in filenames]
    #rozmery obrázkov
    width, height = images[0].size
    combined_image = np.zeros((height, width, 3), dtype=np.float32)
    pixel_counts = np.zeros((height, width), dtype=np.int32)
    #každí obrázok prevedený na array a pridaný do jedného
    for img in images:
        #v prípade že sa na snímkovch vyskytuje biela oblasť je táto oblasť ignorovaná pri kombinovaní
        img_array = np.array(img, dtype=np.float32)
        white_mask = np.all(img_array == [255, 255, 255], axis=-1)
        combined_image[~white_mask] += img_array[~white_mask]
        pixel_counts += ~white_mask

    #vytvorenie kombinovaného obrázka
    combined_image /= np.maximum(pixel_counts[:, :, np.newaxis], 1)

    combined_image = np.clip(combined_image, 0, 255).astype(np.uint8)
    combined_image = Image.fromarray(combined_image)
    #uloženie obrázka
    combined_image.save(os.path.join(output_folder, hour_key + "_jicamarca.png"))


Pri vytváraní kombinovaných obrázkov sme ignorovali biele časti

To spôsobilo že v momente kedy nebolo možné ich nahradiť pri kombinácii sú teraz tieto časti čierne

Pre také obrázky vrátime bielu farbu

In [14]:
#funkcia pre nahrádzanie čiernych pixelov za biele
def replace_black_pixels_with_white(image_path):
    img = Image.open(image_path)
    #rozmery obrázka
    width, height = img.size
    #nahrádzanie pixelov
    for x in range(width):
        for y in range(height):
            pixel = img.getpixel((x, y))

            if pixel == (0, 0, 0) or pixel == (0, 0, 0, 255):
                img.putpixel((x, y), (255, 255, 255))
    #uloženie obrázka
    img.save(image_path)

#funkcia pre aplikovanie na celý priečinok
def process_images_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            image_path = os.path.join(folder_path, filename)

            replace_black_pixels_with_white(image_path)

folder_path = "GOLD_jicamarca_combined"
process_images_in_folder(folder_path)

Obrázky po spojení sme ešte raz orezali do kruhového tvaru

In [15]:
#funkcia pre vytvorenie kruhového výrezu
def crop_to_circle(image):
    #veľkosť obrázka
    width, height = image.size
    radius = min(width, height) // 2
    #vytvorenie kruhovej masky
    circle_mask = Image.new('L', (width, height), 0)
    draw = ImageDraw.Draw(circle_mask)
    draw.ellipse((0, 0, width, height), fill=255)
    #aplikovanie masky
    result = Image.new('RGBA', (width, height))
    result.paste(image, (0, 0), mask=circle_mask)
    return result

#funkcia pre aplikovanie na celý priečinok
def process_images(input_folder):
    for filename in os.listdir(input_folder):
        if filename.endswith('.png'):
            input_path = os.path.join(input_folder, filename)

            with Image.open(input_path) as img:
                cropped_img = crop_to_circle(img)
                cropped_img.save(input_path)

input_folder = 'GOLD_jicamarca_combined'
process_images(input_folder)